In [28]:
import dspy
import helper_dspy
from dspy import teleprompt
from dspy.retrieve.chromadb_rm import ChromadbRM
# from index_bridge_world_system import CHROMADB_DIR, CHROMA_COLLECTION_NAME

import json


class SymbolsToResearch(dspy.Module):
    """
    Extract relevant ticker symbols for analyst doing research
    """
    def __init__(self):
        super().__init__()
        self.entity_extractor = dspy.Predict("instruction, current_thread -> symbols")

    def forward(self, current_thread):
        meta_prompt = f"Your job is to analyze the sequence of user questions and deduce the ticker symbols of companies (if any) that the last question applies to. Be clever, sometimes you will have to deduce the companies from the question because they won't be named. Return only the ticker symbols."
        prediction = self.entity_extractor(
            current_thread=f"{current_thread}"
        )
        answer = prediction.symbols
        if "Symbols: " in answer:
            start = answer.rindex("Symbols: ") + len("Symbols: ")
            answer = answer[start:]
        return [a.strip() for a in answer.split(',')]
    

In [29]:
# Setup DSP
gpt_35_symbol_finder = helper_dspy.get_dspy_client(service_id='gpt_35_turbo', max_tokens=500, temperature=0.0, top_p=0.2)
dspy.settings.configure(lm=gpt_35_symbol_finder)


# Test question
examples = """
Your job is to analyze the sequence of user questions and deduce the ticker symbols of companies (if any) that the last question applies to. Be clever, sometimes you will have to deduce the companies from the question because they won't be named. Return only the ticker symbols.

# Example 1:
Sequence_of_user_questions: 
    - What are the current earnings per share for the big 3 US cloud providers?
Symbols: MSFT,AMZN,GOOG

# Example 2:
Sequence_of_user_questions: 
    - What are the current earnings per share for the big 3 US cloud providers?
    - Can you tell me the definition of opex?
    - Do Tesla and Apple have debt?
Symbols: TSLA,AAPL

# Example 3:
Sequence_of_user_questions: 
    - Did Bank of America increase their earnings last year?
    - How about Wells Fargo?
Symbols: WFC

===

Analyze the sequence of user questions and deduce the ticker symbols of companies (if any) that the last question applies to. Be clever, sometimes you will have to deduce the companies from the question because they won't be named. Return the ticker symbols.

Sequence_of_user_questions:
    - Tell me about the risk of the major US banks.
Symbols: 
"""
module = SymbolsToResearch()
response = module(examples)
print(response)

gpt_35_symbol_finder.history[0]

['N/A']


{'prompt': "Given the fields `current_thread`, produce the fields `symbols`.\n\n---\n\nFollow the following format.\n\nCurrent Thread: ${current_thread}\nSymbols: ${symbols}\n\n---\n\nCurrent Thread: Your job is to analyze the sequence of user questions and deduce the ticker symbols of companies (if any) that the last question applies to. Be clever, sometimes you will have to deduce the companies from the question because they won't be named. Return only the ticker symbols. # Example 1: Sequence_of_user_questions: - What are the current earnings per share for the big 3 US cloud providers? Symbols: MSFT,AMZN,GOOG # Example 2: Sequence_of_user_questions: - What are the current earnings per share for the big 3 US cloud providers? - Can you tell me the definition of opex? - Do Tesla and Apple have debt? Symbols: TSLA,AAPL # Example 3: Sequence_of_user_questions: - Did Bank of America increase their earnings last year? - How about Wells Fargo? Symbols: WFC === Analyze the sequence of user q

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Small training set with question and answer pairs
trainset = [dspy.Example(question="What were the two main things the author worked on before college?", 
                         answer="Writing and programming").with_inputs('question'),
            dspy.Example(question="What kind of writing did the author do before college?", 
                         answer="Short stories").with_inputs('question'),
            ...
            ]

# The teleprompter will bootstrap missing labels: reasoning chains and retrieval contexts
teleprompter = BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

Below is user input to an AI.  

**User inputs**  
Graph the current earnings per share for the US regional banks from 2020 to 2024.

TASK:  
1. Reivew the input and determine if the AI should lookup SEC filings for specific companies to respond to the input.
2. Reframe the input to include the list of companies that might be helpful or reframe the input so it is clear for the AI.
3. Return the reframed input.



===


Below is a sequence of user inputs to an AI.  

**Sequence of user inputs**  
- Can you graph the current earnings per share for the big 3 US cloud providers (Amazon, Microsoft, Google) from 2020 to 2024?
- include apple too

TASK:  

Assume the last input is a follow-up to the previous ones. Reframe the last question so it can stand on its own and return the reframed input.

USER INPUT TO AI:
Graph the current earnings per share for the US regional banks from 2020 to 2023.

TASK:  
Return a comma separated list of ticker symbols the AI should lookup SEC filings for to aid in its response to the input.

RESPONSE FORMAT:
Comma separated list
